# Hypothesis basics - how to write tests

## Getting started

In this session, we are going to start writing some tests using Hypothesis. To have the convenience of running tests in Jupyter notebook. We use [ipytest](https://github.com/chmp/ipytest) to do that.

In [ ]:
import ipytest

ipytest.autoconfig()

As an simple example, this is how an exercises looks like. Can you finish this simple exercise?

In [ ]:
%%ipytest

# write a test adding two integers
def add_two(a: int, b: int) -> int:
    return a+b  # TODO: return the correct value so it passes

# write a simple test for the function
def test_add_two():
    assert add_two(1,2) == 3  # TODO: write the correct assert statement, choose any 2 numbers to test

Now can you write a test using Hypothesis's `given` and [`st.integers`](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.integers) (ignore any warnings)

In [ ]:
%%ipytest

# TODO: import `given` and `strategies` as st
from hypothesis import given, strategies as st

# write a test adding two integers
def add_two(a: int, b: int) -> int:
    return a+b  # TODO: return the correct value so it passes

# write a property-based test for the function
# TODO: use the decorator `given` with proper strategies
@given(st.integers(), st.integers())
def test_add_two(a, b): # TODO: remember to use the strategies
    assert add_two(a,b) == a+b  # TODO: write the correct assert statement

Next, let's make a function that takes a list of integers and add them up.

In [ ]:
%%ipytest

from typing import List

# write a test adding a list of integers
def add_list(nums: List[int]) -> int:
    return sum(nums)  # TODO: return the correct value so it passes

# write a property-based test for the function
# TODO: use the decorator `given` with proper strategies
@given(st.lists(st.integers()))
def test_list(nums): # TODO: remember to use the strategies
    assert add_list(nums) == sum(nums)  # TODO: write the correct assert statement

## Strategies

Strategies are what make Hypothesis so powerful, it generates the testing examples for us and has the [shrinking logics](https://hypothesis.readthedocs.io/en/latest/data.html#shrinking) to keep the example simple. Writing strategies in property-based testing is not a simple task but luckily we have a lot of [strategies available](https://hypothesis.readthedocs.io/en/latest/data.html#) in hypothesis.

In [ ]:
# Look at an example of a strategy
my_st = st.integers()
my_st.example()

In [ ]:
# TODO: pick a few strategies from this list (https://hypothesis.readthedocs.io/en/latest/data.html) and look at the example
my_st = st.decimals()
print(my_st.example())

my_st = st.text()
print(my_st.example())

my_st = st.dates()
print(my_st.example())

## Catching the edge

But having no fails is not fun. Now we see how Hypothesis can catch an edge case

In [ ]:
%%ipytest

# functions that encode and decode a string 
# example from https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.integers
def encode(input_string):
    if input_string == "":
        return ""
    count = 1
    prev = ""
    lst = []
    for character in input_string:
        if character != prev:
            if prev:
                entry = (prev, count)
                lst.append(entry)
            count = 1
            prev = character
        else:
            count += 1
    entry = (character, count)
    lst.append(entry)
    return lst


def decode(lst):
    q = ""
    for character, count in lst:
        q += character * count
    return q

# TODO: write a property-based test for both function

@given(st.text())
def test_decode_inverts_encode(s):
    assert decode(encode(s)) == s

Ah it failed... can you identify in which case it fails? Can you fix it so the test passes?

Since it failed, we would like to make sure Hypothesis will always test that case in the future as it is an edge case that someone may forget about. To do that, we will use [`example`](https://hypothesis.readthedocs.io/en/latest/reproducing.html#hypothesis.example)

In [ ]:
%%ipytest

# TODO: import `example`
from hypothesis import example

@given(st.text())
# TODO: pin the case that failed
@example("")
# TODO: copy your test here
def test_decode_inverts_encode(s):
    assert decode(encode(s)) == s

## Customize strategies

Now go back to the adding number function `add_two` we are going to do something differnt. Instead of taking two integers, we will try tp take two integers as strings.

In [ ]:
%%ipytest

# write a test adding two integers as strings
def add_two(a: str, b: str) -> int:
    return int(a)+int(b)  # TODO: return the correct value so it passes

# write a property-based test for the function
# TODO: use the decorator `given` with proper strategies
@given(st.text(), st.text())
def test_add_two(a, b): # TODO: remember to use the strategies
    assert add_two(a,b) == int(a)+int(b)  # TODO: write the correct assert statement

There are cases that fails. It is because they cannot be casted as integers. Can you think of a way to limit our strategie to just generate test that are numbers? (hint: you can either use the parameters in [st.text](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.text) or use [st.from_regex](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.from_regex)

In [ ]:
%%ipytest

# write a test adding two integers
def add_two(a: str, b: str) -> int:
    return int(a)+int(b)  # TODO: return the correct value so it passes

# write a property-based test for the function
# TODO: use the decorator `given` with proper strategies
only_numbers = st.text(alphabet = st.characters(min_codepoint=48, max_codepoint=57), min_size=1)
@given(only_numbers, only_numbers)
def test_add_two(a, b): # TODO: remember to use the strategies
    assert add_two(a,b) == int(a)+int(b)  # TODO: write the correct assert statement

## Combine strategies

Now, we look back to the first function of `add_two`. We should not be limiting the input to two integers, it can be any number. How about we can accept it to be either integer or float? Then we will need to build our own strategies.

In [ ]:
%%ipytest

# write a test adding two integers
def add_two(a: int|float, b: int|float) -> int:
    return a+b  # TODO: return the correct value so it passes

#TODO: write a target function to be used by `st.builds`
def int_or_float(integer: int, decimal: float, flag: bool):
    if flag:
        return integer
    else:
        return decimal

# write property-based tests for the function

# TODO: use the decorator `given` with proper strategies
number = st.builds(int_or_float, st.integers(), st.decimals(allow_nan=False, allow_infinity=False), st.booleans())
@given(number, number)
def test_add_two(a, b): # TODO: remember to use the strategies
    assert add_two(a,b) == a+b  # TODO: write the correct assert statement

# TODO: write another test to test the sum of the even numbers are still even
even_num = st.builds(lambda x: x*2, st.integers())
@given(even_num, even_num)
def test_add_evens(a, b):
    assert add_two(a,b)%2 == 0

Now you are familar with writing property based tests with hypothesis. Let's look deeper into how to use the extra feature that hypothesis provides for numpy and pandas.